In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from collections import Counter
from tensorflow.keras.optimizers import Adam


C:\Users\Asus\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load your data
train_data = pd.read_excel('C:/Users/Asus/Desktop/New folder (4)/train_test33.xlsx')
test_data = pd.read_csv('C:/Users/Asus/Desktop/New folder (4)/test_test33.csv')

In [3]:
null_indices_test_texts = test_data[test_data['review_description'].isnull()].index
test_data['review_description'].fillna("", inplace=True)
print("Indices of nulls in test_texts:", len(null_indices_test_texts))
print(len(test_data))

Indices of nulls in test_texts: 5
1000


In [4]:
# make target =0 for null rows 
# train_data['rating'] = np.where(train_data['review_description'].isnull(), 0, train_data['rating'])
# train_data['review_description'].fillna("", inplace=True)
train_data.dropna(inplace=True) 

In [5]:
train_texts = train_data['review_description'].tolist()
train_labels = train_data['rating']
test_texts = test_data['review_description']

In [6]:
label_counts = Counter(train_labels)
print(label_counts)


Counter({1: 22989, -1: 13929, 0: 1908})


In [7]:
# Tokenization and Padding
top_words = 25000
max_review_length = 40
embedding_vector_length = 300
num_classes = 3
learning_rate = 0.0001

In [8]:
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_texts)

In [9]:
# # Accessing word indices
# word_index = tokenizer.word_index
# print("Word Index:")
# print(word_index)

# # Accessing word counts
# word_counts = tokenizer.word_counts
# print("\nWord Counts:")
# print(word_counts)

# # Accessing document count
# document_count = tokenizer.document_count
# print("\nDocument Count:")
# print(document_count)

# # Accessing the word-documents dictionary
# word_docs = tokenizer.word_docs
# print("\nWord Documents:")
# print(word_docs)

In [10]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# for i, sequence in enumerate(train_sequences):
#     if 100 < i < 200:
#         print(sequence)
#     elif i >= 200:
#         break  # Stops printing after the 200th sequence
        
          

# j=0
# for i, sequence in enumerate(train_sequences):
#     if sequence==[]:
#        print(train_data['review_description'].iloc[i]+ '  ' + str(train_labels[i]))
#        print(i)
#        j+=1
# print('number_of_null',j)

In [11]:
sequence = [67, 4654, 5471, 108, 2462, 2106, 6830, 1829, 26]

decoded_sentence = " ".join(tokenizer.index_word[idx] for idx in sequence if idx in tokenizer.index_word)
print(decoded_sentence)

اول ضل اتكون الي حكول يتيح لشبكا swvl شكر


In [12]:
train_sequences = pad_sequences(train_sequences, maxlen=max_review_length)
test_sequences = pad_sequences(test_sequences, maxlen=max_review_length)
#  train_sequences
for i, sequence in enumerate(train_sequences):
    if i < 2:
        print(sequence)
    else:
        break  


[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0    39    70   265 10573    98   470    94  4088   265  3422
  1709  6073  1621    36   317   168   301   373   741  5146  1149   315
    27  2203   915   117]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0   10  156  100 1957  513 1411   41  371  172  156 1371]


In [13]:
# Convert labels to categorical format

train_labels = train_labels.astype('category').cat.codes.values

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(train_sequences, train_labels, test_size=0.2, random_state=42, shuffle=True)
train_labels

array([0, 2, 0, ..., 0, 0, 1], dtype=int8)

In [14]:
# Model
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))


# Instantiate the optimizer with the specified learning rate

optimizer = Adam(learning_rate=learning_rate)

# Compile the model, using the customized optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# Train the model with adjusted hyperparameters

print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         7500000   
                                                                 
 conv1d (Conv1D)             (None, None, 32)          28832     
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 32)          0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, None, 100)         53200     
                                                                 
 global_max_pooling1d (Glob  (None, 100)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 3)                

In [15]:

# Training
model.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_val, y_val))
# Prediction on test data
predictions = model.predict(test_sequences)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels


Epoch 1/4


486/486 [==============================] - 34s 67ms/step - loss: 0.7589 - accuracy: 0.6648 - val_loss: 0.5343 - val_accuracy: 0.8187
Epoch 2/4
486/486 [==============================] - 32s 66ms/step - loss: 0.4744 - accuracy: 0.8381 - val_loss: 0.4525 - val_accuracy: 0.8441
Epoch 3/4
486/486 [==============================] - 32s 66ms/step - loss: 0.4067 - accuracy: 0.8653 - val_loss: 0.4405 - val_accuracy: 0.8493
Epoch 4/4
32/32 [==============================] - 0s 3ms/step


array([2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2,
       0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2,
       0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0,
       0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0,
       2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2,
       2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,

In [16]:
def replace_values(lst):
    for i in range(len(lst)):
        if lst[i] == 2:
            lst[i] = 1
        elif lst[i] == 1:
            lst[i] = 0
        elif lst[i] == 0:
            lst[i] = -1
    return lst


In [17]:
predicted_out2=replace_values(predicted_labels)


In [18]:
import csv


# File name for the CSV file
file_name = 'C:/Users/Asus/Desktop/New folder (4)/predicted_ratings3.csv'

# Writing data to a CSV file
with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    writer.writerow(['ID', 'rating'])

    # Write data rows
    for idx, rating in enumerate(predicted_out2, start=1):
        writer.writerow([idx, rating])

print(f"CSV file '{file_name}' has been created.")

CSV file 'C:/Users/Asus/Desktop/New folder (4)/predicted_ratings3.csv' has been created.
